In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import sys
import datetime
from pandas.tseries.offsets import *
from pandas.tseries.offsets import BDay

In [2]:
#this function is used to update data that didn't populate in initial data call above
def update_data(symbol):
    data = yf.download(
            tickers = symbol,
            period = '200d',
            group_by = 'ticker',
            auto_adjust = True,
            threads = True,
            proxy = None)
    return data

In [3]:
stocks = pd.read_csv('cleaned_main_data.csv')


In [4]:
stocks = stocks.drop(stocks.columns[0], axis=1)

In [5]:
stocks.head()

,Symbol,Name,Exchange,Sector,industry,dividendRate,exDividendDate,lastSplitDate,lastSplitFactor,Buy,Sell,Hold,Previous_Earnings_Date,Expected,Actual,Surprise,Future Earnings Date,daysSinceLastDividend
0,DDD,3D Systems Corporation,NYSE,Technology,Computer Software: Prepackaged Software,NaN,0.000000e+00,0,0.0,0.0,0.0,1.0,2020-08-04,-0.1,-0.13,-30,2020-10-28,0.0
1,MMM,3M Company,NYSE,Health Care,Medical/Dental Instruments,5.88,1.597968e+09,0,0.0,1.0,2.0,4.0,2020-07-27,1.8,1.78,-1.17,2020-10-27,-25.0
2,WUBA,58.com Inc.,NYSE,Technology,"Computer Software: Programming, Data Processing",NaN,0.000000e+00,0,0.0,0.0,0.0,0.0,2020-06-25,-0.04,2.09,+4744.44,2020-11-16,0.0
3,EGHT,8x8 Inc,NYSE,Technology,EDP Services,NaN,0.000000e+00,0,0.0,1.0,1.0,6.0,2020-07-29,-0.12,-0.07,+43.55,2020-10-28,0.0
4,EGHT,8x8 Inc,Nasdaq,Technology,EDP Services,NaN,0.000000e+00,0,0.0,1.0,1.0,6.0,2020-07-29,-0.12,-0.07,+43.55,2020-10-28,0.0


In [6]:
# symbols = stocks['Symbol'][:5].tolist()
symbols = stocks['Symbol'].tolist()
type(symbols)

list

In [7]:
len(symbols)

1774

In [8]:
data = yf.download(
tickers = symbols,
period = '200d',
group_by = 'ticker',
auto_adjust = True,
threads = True,
proxy = None)

[*********************100%***********************]  1768 of 1768 completed


In [9]:
data.head(2)

PLRX                        AKTS                            ...  \
           Open High Low Close Volume  Open  High   Low Close  Volume  ...   
Date                                                                   ...   
2019-11-29  NaN  NaN NaN   NaN    NaN  7.63  8.00  7.60  7.63  193300  ...   
2019-12-02  NaN  NaN NaN   NaN    NaN  7.74  7.81  7.44  7.64  192000  ...   

                 AGI                                        PBIO              \
                Open     High       Low     Close   Volume  Open  High   Low   
Date                                                                           
2019-11-29  5.460439  5.63882  5.440618  5.618999  2003000  1.12  1.12  1.10   
2019-12-02  5.589269  5.63882  5.529809  5.609089  1475300  0.80  1.09  0.73   

                         
           Close Volume  
Date                     
2019-11-29  1.10   5900  
2019-12-02  0.84  63500  

[2 rows x 8840 columns]

In [10]:
# stocks['previous_earnings_date'] = pd.to_datetime(stocks['Previous_Earnings_Date'])

In [11]:
# stocks['new_date'] = stocks.previous_earnings_date.map(lambda x: x + 0*BDay())
# stocks.previous_earnings_date = pd.to_datetime(stocks.previous_earnings_date, errors='coerce')

In [12]:
# new_stocks = stocks[~(stocks.Symbol.isin(['GABC', 'FWONA', 'GFED', 'NATI']))]

In [13]:
# new_stocks.shape

In [14]:
stocks.head(2)

,Symbol,Name,Exchange,Sector,industry,dividendRate,exDividendDate,lastSplitDate,lastSplitFactor,Buy,Sell,Hold,Previous_Earnings_Date,Expected,Actual,Surprise,Future Earnings Date,daysSinceLastDividend
0,DDD,3D Systems Corporation,NYSE,Technology,Computer Software: Prepackaged Software,NaN,0.000000e+00,0,0.0,0.0,0.0,1.0,2020-08-04,-0.1,-0.13,-30,2020-10-28,0.0
1,MMM,3M Company,NYSE,Health Care,Medical/Dental Instruments,5.88,1.597968e+09,0,0.0,1.0,2.0,4.0,2020-07-27,1.8,1.78,-1.17,2020-10-27,-25.0


In [15]:
earnings_date_index = stocks.columns.tolist().index('Previous_Earnings_Date')
symbol_index = stocks.columns.tolist().index('Symbol')

In [16]:
earnings_date_index

12

In [17]:
indicators = ['BBP', 'Exponential Moving Average']
for indicator in indicators:
    stocks.loc[:, indicator] = np.nan
stocks.head(1)

,Symbol,Name,Exchange,Sector,industry,dividendRate,exDividendDate,lastSplitDate,lastSplitFactor,Buy,Sell,Hold,Previous_Earnings_Date,Expected,Actual,Surprise,Future Earnings Date,daysSinceLastDividend,BBP,Exponential Moving Average
0,DDD,3D Systems Corporation,NYSE,Technology,Computer Software: Prepackaged Software,NaN,0.0,0,0.0,0.0,0.0,1.0,2020-08-04,-0.1,-0.13,-30,2020-10-28,0.0,NaN,NaN


In [18]:
#calculate exponential moving average
def EMA(data, date, days=5, window=20, smoothing=2):
    data = data / data.iloc[0]
    data['SMA'] = data['Close'].rolling(window=window).mean()
    data['EMA'] = data['SMA']
    
    ema_index = data.columns.tolist().index('EMA')
    value_index = data.columns.tolist().index('Close')
    
    i=0
    for row in data.itertuples():
        
        if i == 0:
            previous_row = row
        else:

            value_today = row[value_index+1]
            ema = (value_today *(smoothing/(1.0+days)) 
                   + previous_row[ema_index+1] *(1.0-(smoothing/(1.0+days))))
            data.at[row[0],'EMA'] = ema
            previous_row = row
            
        i += 1
    idx = data.index.get_loc(date)
    ema = data['EMA'].iloc[idx-days]
    return ema


In [19]:
#calculate Bollinger Bands
def BBP(data, date, days=5, window=20):
    data['SMA'] = data['Close'].rolling(window=window).mean()
    data['SD'] = data['Close'].rolling(window=window).std()
    data['BB_above'] = data['SMA'] + 2*data['SD']
    data['BB_below'] = data['SMA'] - 2*data['SD']
    data['BBP'] = np.divide(data['Close'] - data['SMA'], 2*data['SD'])

    idx = data.index.get_loc(date)
    
    
    bbp = data['BBP'].iloc[idx-days]
    return bbp


In [20]:
#calculate momentum
def mom(data, date, days=5, window=20):
    pass


In [21]:
def dependentVariableCreator(data, date, days=5):

    difference_list=[]
    #five days prior close
    prior_close_date = pd.to_datetime(date)-BDay(days)
    prior_close_date = pd.to_datetime(makeBusinessDay(str(prior_close_date)[0:10]))

    prior_close = data.loc[prior_close_date]['Close']
    
    #closes will keep track of the actual close on a given date
    closes = [prior_close]
    
    for i in range(1,days):
        new_date = pd.to_datetime(prior_close_date) + BDay(i)
        new_date = pd.to_datetime(makeBusinessDay(str(new_date)[0:10]))
        
        new_close = data.loc[new_date]['Close']
        closes.append(new_close)

        difference = new_close-prior_close
        
        if difference >= 0:
            difference_list.append(1)
        else:
            difference_list.append(0)
    return difference_list, closes


In [22]:
def makeBusinessDay(earnings_date):
    
    #the next 3 days rolls holidays over to the next business day
    holidays_2020=['2020-01-01', '2020-01-20', '2020-02-17', '2020-04-10', '2020-05-25', '2020-07-03',
                  '2020-09-07', '2020-11-26', '2020-12-25']
    if earnings_date in holidays_2020:
        earnings_date = str(pd.to_datetime(earnings_date)+BDay())[0:10]
    
    #the following ensures that the day is a business day
    earnings_date = str(pd.to_datetime(earnings_date)+0*BDay())[0:10]
#     print(earnings_date)
    
    return earnings_date


In [24]:
index=0
days=5

for row in stocks.itertuples():
    earnings_date=(row[earnings_date_index+1])
    symbol = row[symbol_index+1]
    
    earnings_date = makeBusinessDay(earnings_date)
    
    if not pd.isnull(earnings_date):
                
        
        try:
            j=0

            while np.isnan(data[symbol]['Open'].loc[earnings_date]):
                # some data wasn't populated.  Check for this and populate it if so

                data[symbol] = update_data(symbol)
                j += 1                     

                if j > 5:
                    print('Data for ', symbol, ' will not update!')
                    break
        except:
#             earnings_date = pd.to_datetime(earnings_date)+BDay()
            print ('Trying with next business day for: ', symbol)
            print('Next Business Day: ', earnings_date)
                    
        #get BBP.  Using try because data may not have updated above!!!
#             stocks.at[index, 'Exponential Moving Average'] == EMA(data[symbol].copy(), earnings_date, days)

        for indicator in indicators:
            try:
                if indicator == 'BBP':
                    stocks.at[index,indicator] = BBP(data[symbol].copy(), earnings_date, days)
                elif indicator == 'Momentum':
                    stocks.at[index,indicator] = mom(data[symbol].copy(), earnings_date, days)
                else:
                    stocks.at[index,indicator] = EMA(data[symbol].copy(), earnings_date, days)
            except:
                print('Could not create ', indicator, ' for ', symbol)
                    
                    
        # create price increase/decrease for each of the 5 days before earnings
        day_list, closes = dependentVariableCreator(data[symbol].copy(), earnings_date,days)
        y_var = [0,1,2,3]
        for var in y_var:
            column = 'day' + str(var+1)
            stocks.at[index, column] = day_list[var]

        #add the actual close for the 5 days prior to earnings
        for i in range(5):
            column = 'close on day ' + str(i)
            stocks.at[index, column] = closes[i]
                    
       
    index +=1


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Data for  BEKE  will not update!
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Data for  LMND  will not update!
[*********************100%***********************]  1 of 1 completed
[*********************100%***********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Data for  SHIPW  will not update!
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Data for  SRGA  will not update!
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%**********

In [25]:
stocks.head(40)

,Symbol,Name,Exchange,Sector,industry,dividendRate,exDividendDate,lastSplitDate,lastSplitFactor,Buy,...,Exponential Moving Average,day1,day2,day3,day4,close on day 0,close on day 1,close on day 2,close on day 3,close on day 4
0,DDD,3D Systems Corporation,NYSE,Technology,Computer Software: Prepackaged Software,NaN,0.000000e+00,0,0.0,0.0,...,0.789753,1.0,0.0,0.0,1.0,6.610000,6.690000,6.590000,6.590000,6.720000
1,MMM,3M Company,NYSE,Health Care,Medical/Dental Instruments,5.88,1.597968e+09,0,0.0,1.0,...,0.937332,1.0,1.0,1.0,1.0,154.959518,156.376633,157.278427,157.853180,158.398224
2,WUBA,58.com Inc.,NYSE,Technology,"Computer Software: Programming, Data Processing",NaN,0.000000e+00,0,0.0,0.0,...,0.841168,1.0,1.0,0.0,0.0,54.220001,54.500000,54.270000,54.040001,54.009998
3,EGHT,8x8 Inc,NYSE,Technology,EDP Services,NaN,0.000000e+00,0,0.0,1.0,...,0.788320,0.0,0.0,0.0,0.0,16.709999,16.639999,16.110001,16.420000,16.549999
4,EGHT,8x8 Inc,Nasdaq,Technology,EDP Services,NaN,0.000000e+00,0,0.0,1.0,...,0.788320,0.0,0.0,0.0,0.0,16.709999,16.639999,16.110001,16.420000,16.549999
5,AHC,A.H. Belo Corporation,NYSE,Consumer Services,Newspapers/Magazines,0.16,1.597277e+09,0,0.0,0.0,...,0.625654,1.0,0.0,0.0,0.0,1.728837,1.787442,1.680000,1.592093,1.572558
6,AOS,A.O Smith Corporation,NYSE,Consumer Durables,Consumer Electronics/Appliances,0.96,1.596067e+09,0,0.0,0.0,...,1.024063,1.0,1.0,1.0,1.0,50.790264,50.969456,51.078960,52.602070,52.004772
7,ATEN,"A10 Networks, Inc.",NYSE,Technology,Computer Communications Equipment,NaN,0.000000e+00,0,0.0,0.0,...,1.085707,0.0,0.0,0.0,0.0,7.670000,7.520000,7.370000,7.450000,7.110000
8,AIR,AAR Corp.,NYSE,Capital Goods,Aerospace,0.30,1.585267e+09,0,0.0,0.0,...,0.454172,1.0,1.0,1.0,1.0,18.940001,19.100000,19.459999,19.180000,19.080000
9,AAN,"Aaron&#39;s, Inc.",NYSE,Technology,Diversified Commercial Services,0.16,1.600214e+09,0,0.0,4.0,...,0.755158,1.0,1.0,1.0,1.0,43.689999,44.430000,44.939999,44.700001,45.970001


In [26]:
stocks.to_csv('completed_features.csv')

In [27]:
day_list, closes = dependentVariableCreator(data['VOLT'].copy(), earnings_date,days)

In [28]:
day_list

[1, 1, 1, 1]

In [29]:
closes

[0.9800000190734863,
 1.100000023841858,
 1.190000057220459,
 1.0800000429153442,
 1.2000000476837158]

In [ ]:
pd.to_datetime('2020-09-07')+0*BDay()